# Preprocessing

Rename Masbaisis_indices_280619.xlsx to Masbasis_indices_280619.xlsx  
Rename Masbasis_2606_2019_color_and_othe_indecies  to Masbasis_260619_color_and_othe_indecies  
Rename/manually change the date format of Masbasis 2021 data from YYMMDD to DDMMYY

In [1]:
%%time

import os
import math
import datetime
import numpy as np
import pandas as pd
from copy import copy

# Dictionaries
import json
from pprint import pprint

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# To display df nicely in loops
from IPython.display import display 
# display(df1.head()) 
# display(df2.head())

# Display rows and columns Pandas
pd.options.display.max_columns = 100
pd.set_option('display.max_rows',100)

Wall time: 2.28 s


In [2]:
# Prints the current working directory
os.getcwd()
# os.listdir()

'C:\\Users\\fahad\\MegaSync\\NMBU\\GitHub\\vPheno'

## Finding Username folder to make general path for multi PC use

In [3]:
username = str(os.getcwd()).split('\\')[2]
username

'fahad'

## ToDo:  
  
2019 2020 data is fine  
Cant use 2017 because of the blue band  
Distt is not normal in 2018 robot  
AREA UNDER THE CURVE in each season for 2018 2017  

## Importing Data

In [4]:
path = r'C:\\Users\\'+username+'\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021'
parent_path = './Data/'
complete_df_path = parent_path+'1. complete/'
incomplete_df_path = parent_path+'1. incomplete/'

# Create export_path folder if not exists already
os.makedirs(complete_df_path, exist_ok=True)
os.makedirs(incomplete_df_path, exist_ok=True)

os.listdir(path)

['2019 Staur Graminor',
 '2019 Vollebekk Graminor',
 '2019 Vollebekk Masbasis',
 '2020 Staur Graminor',
 '2020 Staur Masbasis_bandsNA',
 '2020 Vollebekk Graminor',
 '2020 Vollebekk Masbasis',
 '2020 Vollebekk Robot',
 '2021 Vollebekk Masbasis_yieldNA']

## Data Preparation
### Creating list of all files

In [5]:
# Get the list of all files in directory tree at given path

files_with_address = []
files_list = []

for (dirpath, dirnames, filenames) in os.walk(path):
    files_with_address += [os.path.join(dirpath, file) for file in filenames]
    files_list.extend(filenames)
    
print(len(files_with_address), 'files founnd in the directory')
# files_with_address
# files_list
files_with_address_bkp = copy(files_with_address)

95 files founnd in the directory


## Data Checking/control

### Check for duplicate filenames

In [6]:
print('Total number of files are :', len(files_list))

print('Number of unique file names are:', len(set(files_list)))

print('There is/are', len(files_list) - len(set(files_list)),'duplicate file name/names.')

Total number of files are : 95
Number of unique file names are: 95
There is/are 0 duplicate file name/names.


### Checking if there are multiple sheets in the files

In [7]:
# Print number of sheets in all files
print('The following files have multiple sheets.')

list_multi_sheet = []
for file in files_with_address:
    
    xl_file = pd.ExcelFile(file,engine='openpyxl')
    number_of_sheets = len(xl_file.sheet_names)
    if number_of_sheets > 1:
        print(number_of_sheets, os.path.basename(file), 'in folder', os.path.basename(os.path.dirname(file))
)
        list_multi_sheet.append(file)

The following files have multiple sheets.
3 19BMLFN3 - MASBASIS yield trial Staur 2019.xlsx in folder 2019 Staur Graminor
4 19TvPhenores.xlsx in folder 2019 Vollebekk Graminor
4 19BMLGI1 - MASBASIS yield trial Vollebekk 2019.xlsx in folder 2019 Vollebekk Masbasis
3 Staur-Graminor-Masbasis_2020.xlsx in folder 2020 Staur Graminor
2 20BMLFN3 - MASBASIS avlingsforsøk Staur 2020 lodging data.xlsx in folder 2020 Staur Masbasis_bandsNA
2 Masbasis_2020_staur.xlsx in folder 2020 Staur Masbasis_bandsNA
2 Staur_maturity_heading_yield_2020.xlsx in folder 2020 Staur Masbasis_bandsNA
3 20BMLGI1_2020_tm.xlsx in folder 2020 Vollebekk Masbasis
3 Masbasis_Mica_2020_all_dates_MEDIAN_DP.xlsx in folder 2020 Vollebekk Masbasis
3 ROBOT_2020.xlsx in folder 2020 Vollebekk Robot


### Removing files with multiple sheets from the list

In [8]:
# Removing files with multiple sheets from the list

for i in list_multi_sheet:
    files_with_address.remove(i)
len(files_with_address)

85

### Removing files without dates
(with 2019 in name means they dont have date format)

In [9]:
files_w_2019 = []
for file in files_with_address:
    file_name = os.path.basename(file)
    if '2019' in file_name:
        print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
        files_w_2019.append(file)
    if '2020' in file_name:
        print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
        files_w_2019.append(file)

Staur_Graminor_2019_median PLT918-2050 240719 070819 150819 210819 300819.xlsx in folder 2019 Staur Graminor
Staur_Masbasis_2019 PLT101-866 240719 070819 150819 300819.xlsx in folder 2019 Staur Graminor
2020TGraminor-Vollebekk-res.xlsx in folder 2020 Vollebekk Graminor


In [10]:
# Removing from list
for i in files_w_2019:
    print(i)
    files_with_address.remove(i)

C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\2019 Staur Graminor\Staur_Graminor_2019_median PLT918-2050 240719 070819 150819 210819 300819.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\2019 Staur Graminor\Staur_Masbasis_2019 PLT101-866 240719 070819 150819 300819.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\2020 Vollebekk Graminor\2020TGraminor-Vollebekk-res.xlsx


In [11]:
len(files_with_address)
# files_with_address

82

### Checking number of unique cultivars in the field

In [12]:
# plots_data = pd.read_excel(files_with_address[0],engine='openpyxl')
# # Pandas converts 'NA' string to NaN. Need to change those to 
# # some string to get a count as NaNs are not counted as unique values

# plots_data.Name.fillna('-', inplace=True)
# plots_data.CodeName.fillna('-', inplace=True)

# # Creating a new column as multiple plots were named 'NA' but the 
# # CodeName was different for each one of them
# plots_data['NameCode'] = plots_data.Name+plots_data.CodeName

# plots_data
# len(plots_data.NameCode.unique())
# plots_data.NameCode.value_counts()
# # plots_data.NameCode.value_counts().sum()
# # plots_data

## Importing data files to Pandas

In [13]:
%%time

all_df = []
for data in files_with_address:
    file_name = os.path.splitext(os.path.basename(data))[0]

    # Replce all invalid characters in the name
    file_name = file_name.replace("-", "_")
    file_name = file_name.replace(" ", "_")
    file_name = file_name.replace("(", "")
    file_name = file_name.replace(")", "")
    df_name = file_name.replace(".", "")
    # Test: Check if the same date is already present in the current dict key
    if df_name in all_df:
        print(f'A file with the same name {df_name} has already been imported. \n Please check if there is duplication of data.')
        raise NameError
            
    all_df.append(df_name)

    locals()[df_name] = pd.read_excel(data, engine='openpyxl')
    print(df_name, '=====', locals()[df_name].shape)

Staur_070819_mean_median_SP ===== (1328, 51)
Staur_150819_mean_median_SP ===== (1328, 51)
Staur_210819_mean_median_SP ===== (1328, 51)
Staur_240719_mean_median_SP ===== (1328, 51)
Staur_300819_mean_median_SP ===== (1328, 51)
Graminor_070819_2 ===== (600, 49)
Graminor_150819 ===== (600, 49)
Graminor_eastwest_050819 ===== (600, 49)
Graminor_250719 ===== (600, 46)
Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted ===== (597, 49)
Graminor_150719 ===== (600, 49)
Graminor_220719_corrected ===== (600, 49)
Graminor_eastwest_020719_NIR_half_missing ===== (600, 22)
Graminor_east_110719 ===== (300, 49)
Graminor_060619 ===== (600, 49)
Graminor_110619 ===== (600, 49)
Graminor_280619_corrected ===== (600, 49)
Graminor_eastwest_250619_correct_duplicate_east ===== (600, 97)
Masbasis_050719_corrected ===== (528, 49)
Masbasis_060619_Indices ===== (528, 56)
Masbasis_070819_correct ===== (528, 49)
Masbasis_110619 ===== (528, 82)
Masbasis_150719 ===== (528, 49)
Masbasis_220719_correct ===== (

In [14]:
print(f'Total imported {len(all_df)}')

Total imported 82


## Data Control: Check which df have the data column heading we need

### Listing column headings into df

In [25]:
%%time

# Finding max number of columns in all df

len_columns = []
for df in all_df:
    cols_df = locals()[df].columns
    len_columns.append(len(cols_df))
max_cols_in_df = max(len_columns)

# Now creating a empty df to collect all column headings
columns_df = pd.DataFrame(data=range(0,max_cols_in_df), columns = ['ID'])
columns_df.drop('ID', axis=1, inplace=True)

for df in all_df:
    cols_df = locals()[df].columns
    columns_df[df] = pd.Series(cols_df)
columns_df

Wall time: 73.2 ms


,Staur_070819_mean_median_SP,Staur_150819_mean_median_SP,Staur_210819_mean_median_SP,Staur_240719_mean_median_SP,Staur_300819_mean_median_SP,Graminor_070819_2,Graminor_150819,Graminor_eastwest_050819,Graminor_250719,"Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted",Graminor_150719,Graminor_220719_corrected,Graminor_eastwest_020719_NIR_half_missing,Graminor_east_110719,Graminor_060619,Graminor_110619,Graminor_280619_corrected,Graminor_eastwest_250619_correct_duplicate_east,Masbasis_050719_corrected,Masbasis_060619_Indices,Masbasis_070819_correct,Masbasis_110619,Masbasis_150719,Masbasis_220719_correct,Masbasis_260619_color_and_othe_indecies,Masbasis_290719,Masbasis_indices_280619,Staur_090720_M,Staur_160720_Mica,Staur_200620_Mica_index,Staur_240720,Staur_250620_Mica_index,Staur_310720,Graminor_eastwest_040720,Graminor_eastwest_040820,Graminor_eastwest_070720_correct,Graminor_eastwest_090720_duplication_same_as_010720,Graminor_eastwest_130720,Graminor_eastwest_140820,Graminor_eastwest_300720,Graminor_east_010720,Graminor_east_170720cpg,Graminor_east_180620,Graminor_east_200720,Graminor_Mica_eastcorrect_west_240620,Masbasis_Mica_070820,Masbasis_Mica_120820,Masbasis_Mica_140820,Masbasis_Mica_010720,Masbasis_Mica_080720,Masbasis_Mica_130720,Masbasis_mica_170720,Masbasis_Mica_220720,"Masbasis_Mica_300720_duplicate_plots_deleted_1332,1329,1330,1331same_data",Masbasis_mica_180620_several_missing_rows_deleted,Masbasis_mica_240620,Masbasis_Mica_260620,Robot_Mica_010720,Robot_Mica_040820,Robot_Mica_070720,Robot_Mica_070820,Robot_Mica_120820,Robot_Mica_130720,Robot_Mica_140820,Robot_Mica_160720,Robot_mica_180620,Robot_Mica_200720,Robot_Mica_220720,Robot_Mica_230620,Robot_Mica_240620,Robot_Mica_250620,Robot_Mica_270720,Robot_Mica_290620,Robot_Mica_300720,Masbasis_mica_020621,Masbasis_mica_090721_excel,Masbasis_mica_130721,Masbasis_mica_170621,Masbasis_mica_220721,Masbasis_mica_230621_excel,Masbasis_mica_240621_excel,Masbasis_mica_260721
0,Unnamed: 0,Unnamed: 0,Unnamed: 0,Unnamed: 0,Unnamed: 0,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Unnamed: 0,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID
1,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Plot_ID,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,_Bluemean,Blue_Mean_Value,Plot_ID,Blue_Mean_Value,_Bluemean,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,_Bluemea_1,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,_Bluecount,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,_Bluecount,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,_Bluemean,Blue_Mean_Value,Blue_Mean_Value,_Bluemean,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value
2,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Mean_Value,Blue_Median_Value,Blue_Median_Value,Blue_M

In [17]:
columns_df_

,Staur_070819_mean_median_SP,Staur_150819_mean_median_SP,Staur_210819_mean_median_SP,Staur_240719_mean_median_SP,Staur_300819_mean_median_SP,Graminor_070819_2,Graminor_150819,Graminor_eastwest_050819,Graminor_250719,"Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted",Graminor_150719,Graminor_220719_corrected,Graminor_eastwest_020719_NIR_half_missing,Graminor_east_110719,Graminor_060619,Graminor_110619,Graminor_280619_corrected,Graminor_eastwest_250619_correct_duplicate_east,Masbasis_050719_corrected,Masbasis_060619_Indices,Masbasis_070819_correct,Masbasis_110619,Masbasis_150719,Masbasis_220719_correct,Masbasis_260619_color_and_othe_indecies,Masbasis_290719,Masbasis_indices_280619,Staur_090720_M,Staur_160720_Mica,Staur_200620_Mica_index,Staur_240720,Staur_250620_Mica_index,Staur_310720,Graminor_eastwest_040720,Graminor_eastwest_040820,Graminor_eastwest_070720_correct,Graminor_eastwest_090720_duplication_same_as_010720,Graminor_eastwest_130720,Graminor_eastwest_140820,Graminor_eastwest_300720,Graminor_east_010720,Graminor_east_170720cpg,Graminor_east_180620,Graminor_east_200720,Graminor_Mica_eastcorrect_west_240620,Masbasis_Mica_070820,Masbasis_Mica_120820,Masbasis_Mica_140820,Masbasis_Mica_010720,Masbasis_Mica_080720,Masbasis_Mica_130720,Masbasis_mica_170720,Masbasis_Mica_220720,"Masbasis_Mica_300720_duplicate_plots_deleted_1332,1329,1330,1331same_data",Masbasis_mica_180620_several_missing_rows_deleted,Masbasis_mica_240620,Masbasis_Mica_260620,Robot_Mica_010720,Robot_Mica_040820,Robot_Mica_070720,Robot_Mica_070820,Robot_Mica_120820,Robot_Mica_130720,Robot_Mica_140820,Robot_Mica_160720,Robot_mica_180620,Robot_Mica_200720,Robot_Mica_220720,Robot_Mica_230620,Robot_Mica_240620,Robot_Mica_250620,Robot_Mica_270720,Robot_Mica_290620,Robot_Mica_300720,Masbasis_mica_020621,Masbasis_mica_090721_excel,Masbasis_mica_130721,Masbasis_mica_170621,Masbasis_mica_220721,Masbasis_mica_230621_excel,Masbasis_mica_240621_excel,Masbasis_mica_260721
0,Unnamed: 0,Unnamed: 0,Unnamed: 0,Unnamed: 0,Unnamed: 0,id,id,id,id,id,id,id,id,id,id,id,id,id,plot,Unnamed: 0,plot,plot,plot,plot,plot,plot,plot,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id
1,Plot,Plot,Plot,Plot,Plot,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_bluemean,_Bluemea_1,_Bluemean,_Bluemea_2,_Bluemea_1,_Bluemea_1,_Bluemean,_Bluemean,_Bluemean,plot_June 6th,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemea_1,_Bluemea_1,blue_mean,blue_mean,blue_mean,blue_mean,blue_mean,blue_mean,_Bluemean,_Bluemean,_Bluemean,_Bluemea_1,_Bluemean,_Bluemean,_Bluemean,_Bluecount,_Bluemea_2,_BLuemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemea_1,_Bluemean,_Bluecount,_Bluemean,_Bluemean,_Bluemean,_Bluemea_1,_Bluemean,_Bluemea_1,_Bluemean,_Bluemea_2,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemean,_Bluemea_2,_Bluemean,_Bluemea_1,_Bluemean,_Bluemean,_Bluemean,blue_mean,blue_mean,blue_mean,blue_mean,blue_mean,blue_mean,blue_mean,blue_mean
2,Blue_median,Blue_median,Blue_median,Blue_median,Blue_median,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_bluemedia,_Bluemed_1,_Bluemedia,_Bluemed_2,_Bluemed_1,_Bluemed_1,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemean_June 6th,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemed_1,_Bluemed_1,blue_median,blue_median,blue_median,blue_median,blue_median,blue_median,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluesum,_Bluemed_1,_BLuemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemed_1,_Bluemedia,_Bluemean,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemed_1,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemed_1,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemedia,_Bluemed_2,_Bluemedia,_Bluemed_1,_Bluemedia,_Bluemedia,_Bluemedia,blue_median,blue_median,blue_median,blue_median,blue_median,blue_m

In [53]:
cols_compare

,Staur_070819_mean_median_SP,Staur_150819_mean_median_SP,Staur_210819_mean_median_SP,Staur_240719_mean_median_SP,Staur_300819_mean_median_SP,Graminor_070819_2,Graminor_150819,Graminor_eastwest_050819,Graminor_250719,"Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted",Graminor_150719,Graminor_220719_corrected,Graminor_eastwest_020719_NIR_half_missing,Graminor_east_110719,Graminor_060619,Graminor_110619,Graminor_280619_corrected,Graminor_eastwest_250619_correct_duplicate_east,Masbasis_050719_corrected,Masbasis_060619_Indices,Masbasis_070819_correct,Masbasis_110619,Masbasis_150719,Masbasis_220719_correct,Masbasis_260619_color_and_othe_indecies,Masbasis_290719,Masbasis_indices_280619,Staur_090720_M,Staur_160720_Mica,Staur_200620_Mica_index,Staur_240720,Staur_250620_Mica_index,Staur_310720,Graminor_eastwest_040720,Graminor_eastwest_040820,Graminor_eastwest_070720_correct,Graminor_eastwest_090720_duplication_same_as_010720,Graminor_eastwest_130720,Graminor_eastwest_140820,Graminor_eastwest_300720,Graminor_east_010720,Graminor_east_170720cpg,Graminor_east_180620,Graminor_east_200720,Graminor_Mica_eastcorrect_west_240620,Masbasis_Mica_070820,Masbasis_Mica_120820,Masbasis_Mica_140820,Masbasis_Mica_010720,Masbasis_Mica_080720,Masbasis_Mica_130720,Masbasis_mica_170720,Masbasis_Mica_220720,"Masbasis_Mica_300720_duplicate_plots_deleted_1332,1329,1330,1331same_data",Masbasis_mica_180620_several_missing_rows_deleted,Masbasis_mica_240620,Masbasis_Mica_260620,Robot_Mica_010720,Robot_Mica_040820,Robot_Mica_070720,Robot_Mica_070820,Robot_Mica_120820,Robot_Mica_130720,Robot_Mica_140820,Robot_Mica_160720,Robot_mica_180620,Robot_Mica_200720,Robot_Mica_220720,Robot_Mica_230620,Robot_Mica_240620,Robot_Mica_250620,Robot_Mica_270720,Robot_Mica_290620,Robot_Mica_300720,Masbasis_mica_020621,Masbasis_mica_090721_excel,Masbasis_mica_130721,Masbasis_mica_170621,Masbasis_mica_220721,Masbasis_mica_230621_excel,Masbasis_mica_240621_excel,Masbasis_mica_260721
0,Unnamed: 0Unnamed: 0,Unnamed: 0Unnamed: 0,Unnamed: 0Unnamed: 0,Unnamed: 0Unnamed: 0,Unnamed: 0Unnamed: 0,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDplot,Unnamed: 0Unnamed: 0,Plot_IDplot,Plot_IDplot,Plot_IDplot,Plot_IDplot,Plot_IDplot,Plot_IDplot,Plot_IDplot,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid,Plot_IDid
1,Plot_IDPlot,Plot_IDPlot,Plot_IDPlot,Plot_IDPlot,Plot_IDPlot,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_bluemean,Blue_Mean_Value_Bluemea_1,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemea_2,Blue_Mean_Value_Bluemea_1,Blue_Mean_Value_Bluemea_1,Blue_Mean_Value_Bluemean,_Bluemean_Bluemean,Blue_Mean_Value_Bluemean,Plot_IDplot_June 6th,Blue_Mean_Value_Bluemean,_Bluemean_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemea_1,Blue_Mean_Value_Bluemea_1,Blue_Mean_Valueblue_mean,Blue_Mean_Valueblue_mean,Blue_Mean_Valueblue_mean,Blue_Mean_Valueblue_mean,Blue_Mean_Valueblue_mean,Blue_Mean_Valueblue_mean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,_Bluemea_1_Bluemea_1,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,_Bluecount_Bluecount,Blue_Mean_Value_Bluemea_2,Blue_Mean_Value_BLuemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blue_Mean_Value_Bluemean,Blu

In [52]:
cols_compare = columns_df+columns_df_
for cols in cols_compare.columns:
    print('******************************', cols, '******************************')
    print('*****', 'median', '*************************')
    print(*[x for x in cols_compare[cols].to_list() if 'median' in str(x)], sep = "\n")
    print('*****', 'mean', '*************************')
    print(*[x for x in cols_compare[cols].to_list() if 'mean' in str(x)], sep = "\n")
    print('*****', 'st', '*************************')
    print(*[x for x in cols_compare[cols].to_list() if 'std' in str(x)], sep = "\n")
    
cols_compare[cols].to_list()

****************************** Staur_070819_mean_median_SP ******************************
***** median *************************
Blue_Mean_ValueBlue_median
Green_Mean_ValueGreen_median
Red_Mean_ValueRed_median
RedEdge_Mean_ValueRedEdge_median
NIR_Mean_ValueNIR_median
NDVI_medianNDVI_median
MTCI_medianMTCI_median
EVI_medianEVI_median
DVI_medianDVI_median
RVI_medianRVI_median
VARI_medianVARI_median
EXG_medianEXG_median
EXGR_medianEXGR_median
GLI_medianGLI_median
GNDVI_medianGNDVI_median
GVI_medianGVI_median
***** mean *************************
Blue_Median_ValueBlue_mean
Green_Median_ValueGreen_mean
Red_Median_ValueRed_mean
RedEdge_Median_ValueRedEdge_mean
NIR_Median_ValueNIR_mean
NDVI_meanNDVI_mean
MTCI_meanMTCI_mean
EVI_meanEVI_mean
DVI_meanDVI_mean
RVI_meanRVI_mean
VARI_meanVARI_mean
EXG_meanEXG_mean
EXGR_meanEXGR_mean
GLI_meanGLI_mean
GNDVI_meanGNDVI_mean
GVI_meanGVI_mean
***** st *************************
Blue_Std_Dev_ValueBlue_std
Green_Std_Dev_ValueGreen_std
Red_Std_Dev_ValueRed_st

***** median *************************
_DVImedian_DVImedian
_EVImedian_EVImedian
_EXGmedian_EXGmedian
_GLImedian_GLImedian
_GVImedian_GVImedian
NIR_Median_Value_NIRmedian
Red_Median_Value_Redmedian
RedEdge_Median_Value_RedEdgemedian
_RVImedian_RVImedian
***** mean *************************
Blue_Mean_Value_Bluemean
_DVImean_DVImean
_EVImean_EVImean
_EXGmean_EXGmean
_EXGRmean_EXGRmean
_GLImean_GLImean
_GNDVImean_GNDVImean
Green_Mean_Value_Greenmean
_GVImean_GVImean
_MTCImean_MTCImean
_NDVImean_NDVImean
NIR_Mean_Value_NIRmean
Red_Mean_Value_Redmean
RedEdge_Mean_Value_RedEdgemean
_RVImean_RVImean
_VARImean_VARImean
***** st *************************
Blue_Std_Dev_Value_Bluestdev
_DVIstdev_DVIstdev
_EVIstdev_EVIstdev
_EXGstdev_EXGstdev
_EXGRstdev_EXGRstdev
_GLIstdev_GLIstdev
_GNDVIstde_GNDVIstde
Green_Std_Dev_Value_Greenstde
_GVIstdev_GVIstdev
_MTCIstdev_MTCIstdev
_NDVIstdev_NDVIstdev
NIR_Std_Dev_Value_NIRstdev
Red_Std_Dev_Value_Redstdev
RedEdge_Std_Dev_Value_RedEdgestd
_RVIstdev_RVIstdev
_V

['Plot_IDid',
 'Blue_Mean_Valueblue_mean',
 'Blue_Median_Valueblue_median',
 'Blue_Std_Dev_Valueblue_stdev',
 'Green_Mean_Valuegreen_mean',
 'Green_Median_Valuegreen_median',
 'Green_Std_Dev_Valuegreen_stdev',
 'ndvi_meanndvi_mean',
 'ndvi_medianndvi_median',
 'ndvi_stdevndvi_stdev',
 'NIR_Mean_Valuenir_mean',
 'NIR_Median_Valuenir_median',
 'NIR_Std_Dev_Valuenir_stdev',
 'Red_Mean_Valuered_mean',
 'Red_Median_Valuered_median',
 'Red_Std_Dev_Valuered_stdev',
 'RedEdge_Mean_Valuerededge_mean',
 'RedEdge_Median_Valuerededge_median',
 'RedEdge_Std_Dev_Valuerededge_stdev',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

### Convert columns_df to dictoionary

In [23]:
%%time

# Function to convert df to dict while dropping nan in each column separately

def comp_dropna(df1):
    return {k: v.dropna().to_dict() for k,v in df1.items()}

columns_dict = comp_dropna(columns_df)
# columns_dict

Wall time: 26.1 ms


### Finding Column heading with the text we are looking for in

In [24]:
%%time

print(f'Total {len(all_df)} datasets have been imported.')

# Find columns with key from the following list
# Rename the founc columns to the values against the keys

id_cols_var = ['id', 'plot']
id_cols_new = ['Plot_ID']


find_cols = {
    id_cols_var[0]: id_cols_new[0],
    id_cols_var[1]: id_cols_new[0],
    'Blue': 'Blue',
    'Gre': 'Green',
    'Red': 'Red',
    'RedE': 'RedEdge',
    'NIR': 'NIR',
#     'NDVI': 'NDVI',
#     'MTCI': 'MTCI'
}


dtype_f = {0: '_Mean_Value', 1: '_Median_Value', 2: '_Std_Dev_Value'}
num_std_dtype_colunms = len(dtype_f)

list_ok_df_per_query = []
list_problem_df_per_query = []

for query, query_f in find_cols.items():
    
    # First, we loop through all df to find the max of number of relavant columns in any df
    num_col_found_all = []
    for df, cols_dict in columns_dict.items():

        # List to save the relavant columns found in the df
        cols_found = []
        
        # Loop through all column names
        for key, item in cols_dict.items():
            
            # Check if required text is in colmn name,i.e. item
            
            # Ensure first that the text does not contain 'count' and 'sum' in it
            if item.lower().find('sum'.lower()) == -1 and item.lower().find('count'.lower()) == -1:
            
                # When searching for Red, search for red but ignore RedEdge,i.e. RedE
                if query.lower()=='red':
                    if item.lower().find(query.lower()) != -1 and item.lower().find('RedE'.lower()) == -1:
                        cols_found.append(item)

                # For the rest also checking we do not get 'GNDVI' columns. Don't neet them yet
                elif item.lower().find(query.lower()) != -1 and item.lower().find('GNDVI'.lower()) == -1:
                    cols_found.append(item)

        num_col_found_all.append(len(cols_found))
    max_cols = max(num_col_found_all)
#     print(query, max_cols)

    # Define a df for saving the columns found if num of col is not num_std_dtype_colunms
    temp_df_non_std_cols = 'col_df_non_std_cols_'+query
    
    # Add a condition in data if no columns found, then still create a df with 1 column
    locals()[temp_df_non_std_cols] = pd.DataFrame(data=range(0,max_cols) if max_cols > 1 else range(0,1),
                                                  columns = [query])
    list_problem_df_per_query.append(temp_df_non_std_cols)
    
    # Define a df for saving the columns found if num of col is num_std_dtype_colunms
    temp_df_std_cols= 'col_df_std_cols_'+query
    locals()[temp_df_std_cols] = pd.DataFrame(data=range(0,num_std_dtype_colunms), columns = [query])
    list_ok_df_per_query.append(temp_df_std_cols)
    
    
    
    # Now we again loop and append those columns to a relavant df for each query
    
    for df, cols_dict in columns_dict.items():

        # List to save the relavant columns found in the df
        cols_found = []

        # Loop through all column names
        for key, item in cols_dict.items():

            # Check if required text is in colmn name,i.e. item
            
            # Ensure first that the text does not contain 'count' and 'sum' in it
            if item.lower().find('sum'.lower()) == -1 and item.lower().find('count'.lower()) == -1 and item.lower().find('GreenND'.lower()) == -1:
            
                # When searching for Red, search for red but ignore RedEdge,i.e. RedE
                if query.lower()=='red':
                    if item.lower().find(query.lower()) != -1 and item.lower().find('RedE'.lower()) == -1:
                        cols_found.append(item)

                # For the rest also checking we do not get 'GNDVI' columns. Don't neet them yet
                elif item.lower().find(query.lower()) != -1 and item.lower().find('GNDVI'.lower()) == -1:
                    cols_found.append(item)

        # Adding the found values to a column in dataframe
        
        # For non standard columns
        if len(cols_found) != num_std_dtype_colunms and query not in id_cols_var:
            locals()[temp_df_non_std_cols][df] = pd.Series(cols_found, dtype='str')
        
        # For standrd columns
        if len(cols_found) == num_std_dtype_colunms or query in id_cols_var:
            locals()[temp_df_std_cols][df] = pd.Series(cols_found, dtype='str')
            

        if query in id_cols_var:
            num_id_cols = 0
            for temp_query in id_cols_var:
                for column in cols_dict.values():
                    if column.lower().find(temp_query.lower()) == 0:
                        num_id_cols += 1
            if num_id_cols !=1:
                if num_id_cols == 0:
                    print(query, 'Error: ID column not found in' ,df)
                else:
                    print('Error: More than one ID column found in ' ,df)
    
    # Dropping the empty query named column from the columns dataframe
    # For non standard columns
    locals()[temp_df_non_std_cols].drop(query, axis=1, inplace=True)
    # For standrd columns
    locals()[temp_df_std_cols].drop(query, axis=1, inplace=True)

    
    # Drop extra empty rows if the query is id or plot
    # Drop empty columns, where the id is different than the one being checked
    # Drop first column with 
    # Assumption: There is only one id columns in each df
    # Assumption being checked in previous loop
    if query in id_cols_var:
        locals()[temp_df_std_cols].drop([1, 2], axis=0, inplace=True)
        locals()[temp_df_std_cols].dropna(axis=1, inplace=True)
    
    
    # ===========================Start_of_Analysis_1================================
    # Analysis Part: Uncomment for new data
    # Confirm if all row elements corresopnd to data aggregation types in the dict
    
    
    agg_types_dict = {0:'mea', 1:'med', 2:'st'}
    
    if query in id_cols_var:
        pass
    else:
        for key, agg_type in agg_types_dict.items():
            check_df = locals()[temp_df_std_cols]#.drop(query, axis=1)
            for row_item in check_df.iloc[key].tolist():
                # If aggregation type not found in the column name, then print the following
                if str(row_item).lower().find(agg_type) == -1:
                    print(f'{row_item}, {key}, {agg_type}')
        print('If the printed df has 3 rows with no missing values for base indices, then all is good.')
    print(query, locals()[temp_df_std_cols].shape)
    display(locals()[temp_df_std_cols])
    print('Missing values: ',locals()[temp_df_std_cols].isna().sum().sum())
    
    
    # Analysis shows that for df with num_std_dtype_colunms relavant column names; 
    # green and RedEdge columns have naming problems where the agg_types are
    # not the same for mea and med for some green columns
    # and mea, med, and st for most of the RedEdge columns
    # =============================End_of_Analysis_1================================





    # ===========================Start_of_Analysis_2================================
    # Analysis Part: Uncomment for new data
    # Renaming the column names to standard names 
    # Renaming based on the rule that the first name is mean, second is median and 
    # *************THIS RULE IS WRONG ***************
    # the third is standard deviation.
    # This assumption has been partially checked and confirmed in Analysis_1
    
    # The id column has to be dealed separately since it does not have 
    # num_std_dtype_colunms variations i.e. mean, median, and std_dev
    df_found_correct_cols = locals()[temp_df_std_cols].shape[1]
    renamed_count = 0
    if query in id_cols_var:
        for x, y in locals()[temp_df_std_cols].items():
            locals()[x].rename({y[0]: query_f}, axis=1, inplace=True)
#             print(f'Renamed {query_f} column for dataframe {x} against value {y[0]}')
            renamed_count += 1

    else:
        for x, y in locals()[temp_df_std_cols].items():
            locals()[x].rename({y[0]: query_f+dtype_f[0], y[1]: query_f+dtype_f[1],\
                                y[2]: query_f+dtype_f[2]}, axis=1, inplace=True)
#             print(f'Renamed {query_f} columns for dataframe {x}')
            renamed_count += 1
    print(f'{query} => {query_f} column renamed in {renamed_count} of {df_found_correct_cols} found dataframes')
#     display(locals()[temp_df_std_cols].shape[2])
    
    # =============================End_of_Analysis_2================================

    
    
    
    
    
    
    # ===========================Start_of_Analysis_3================================
    # Analysis Part: Uncomment for new data
    # Checking the uncommon column names/combination or df with extra columns that 
    # make finding the correct data confusing

    print(temp_df_non_std_cols)
    display(locals()[temp_df_non_std_cols])

    # =============================End_of_Analysis_3================================
    




Total 82 datasets have been imported.
id (1, 68)


,Graminor_070819_2,Graminor_150819,Graminor_eastwest_050819,Graminor_250719,"Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted",Graminor_150719,Graminor_220719_corrected,Graminor_eastwest_020719_NIR_half_missing,Graminor_east_110719,Graminor_060619,Graminor_110619,Graminor_280619_corrected,Graminor_eastwest_250619_correct_duplicate_east,Staur_090720_M,Staur_160720_Mica,Staur_200620_Mica_index,Staur_240720,Staur_250620_Mica_index,Staur_310720,Graminor_eastwest_040720,Graminor_eastwest_040820,Graminor_eastwest_070720_correct,Graminor_eastwest_090720_duplication_same_as_010720,Graminor_eastwest_130720,Graminor_eastwest_140820,Graminor_eastwest_300720,Graminor_east_010720,Graminor_east_170720cpg,Graminor_east_180620,Graminor_east_200720,Graminor_Mica_eastcorrect_west_240620,Masbasis_Mica_070820,Masbasis_Mica_120820,Masbasis_Mica_140820,Masbasis_Mica_010720,Masbasis_Mica_080720,Masbasis_Mica_130720,Masbasis_mica_170720,Masbasis_Mica_220720,"Masbasis_Mica_300720_duplicate_plots_deleted_1332,1329,1330,1331same_data",Masbasis_mica_180620_several_missing_rows_deleted,Masbasis_mica_240620,Masbasis_Mica_260620,Robot_Mica_010720,Robot_Mica_040820,Robot_Mica_070720,Robot_Mica_070820,Robot_Mica_120820,Robot_Mica_130720,Robot_Mica_140820,Robot_Mica_160720,Robot_mica_180620,Robot_Mica_200720,Robot_Mica_220720,Robot_Mica_230620,Robot_Mica_240620,Robot_Mica_250620,Robot_Mica_270720,Robot_Mica_290620,Robot_Mica_300720,Masbasis_mica_020621,Masbasis_mica_090721_excel,Masbasis_mica_130721,Masbasis_mica_170621,Masbasis_mica_220721,Masbasis_mica_230621_excel,Masbasis_mica_240621_excel,Masbasis_mica_260721
0,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id,id


Missing values:  0
id => Plot_ID column renamed in 68 of 68 found dataframes
col_df_non_std_cols_id


""
0


plot (1, 14)


...

NIR_median, 0, mea
NIR_median, 0, mea
NIR_median, 0, mea
NIR_median, 0, mea
NIR_median, 0, mea
NIR_mean, 1, med
NIR_mean, 1, med
NIR_mean, 1, med
NIR_mean, 1, med
NIR_mean, 1, med
NIR_June 6th, 1, med
If the printed df has 3 rows with no missing values for base indices, then all is good.
NIR (3, 76)


,Staur_070819_mean_median_SP,Staur_150819_mean_median_SP,Staur_210819_mean_median_SP,Staur_240719_mean_median_SP,Staur_300819_mean_median_SP,Graminor_070819_2,Graminor_150819,Graminor_eastwest_050819,Graminor_250719,"Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted",Graminor_150719,Graminor_eastwest_020719_NIR_half_missing,Graminor_east_110719,Graminor_060619,Graminor_110619,Graminor_280619_corrected,Masbasis_050719_corrected,Masbasis_060619_Indices,Masbasis_070819_correct,Masbasis_150719,Masbasis_220719_correct,Masbasis_260619_color_and_othe_indecies,Masbasis_290719,Masbasis_indices_280619,Staur_090720_M,Staur_160720_Mica,Staur_200620_Mica_index,Staur_240720,Staur_250620_Mica_index,Staur_310720,Graminor_eastwest_040720,Graminor_eastwest_040820,Graminor_eastwest_070720_correct,Graminor_eastwest_130720,Graminor_eastwest_140820,Graminor_eastwest_300720,Graminor_east_010720,Graminor_east_170720cpg,Graminor_east_180620,Graminor_east_200720,Graminor_Mica_eastcorrect_west_240620,Masbasis_Mica_070820,Masbasis_Mica_120820,Masbasis_Mica_140820,Masbasis_Mica_010720,Masbasis_Mica_080720,Masbasis_Mica_130720,Masbasis_mica_170720,Masbasis_Mica_220720,"Masbasis_Mica_300720_duplicate_plots_deleted_1332,1329,1330,1331same_data",Masbasis_mica_180620_several_missing_rows_deleted,Masbasis_mica_240620,Masbasis_Mica_260620,Robot_Mica_010720,Robot_Mica_040820,Robot_Mica_070720,Robot_Mica_120820,Robot_Mica_130720,Robot_Mica_140820,Robot_mica_180620,Robot_Mica_200720,Robot_Mica_220720,Robot_Mica_230620,Robot_Mica_240620,Robot_Mica_250620,Robot_Mica_270720,Robot_Mica_290620,Robot_Mica_300720,Masbasis_mica_020621,Masbasis_mica_090721_excel,Masbasis_mica_130721,Masbasis_mica_170621,Masbasis_mica_220721,Masbasis_mica_230621_excel,Masbasis_mica_240621_excel,Masbasis_mica_260721
0,NIR_median,NIR_median,NIR_median,NIR_median,NIR_median,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean_2,_NIRmean_1,_NIRmean_1,_NIRmean,_NIRmean,_NIRmean_June 6th,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean_1,nir_mean,nir_mean,nir_mean,nir_mean,nir_mean,nir_mean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean_1,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean_1,_NIRmean,_NIRmean,_NIRmean_1,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean,_NIRmean_2,_NIRmean,_NIRmean_1,_NIRmean,_NIRmean,_NIRmean,nir_mean,nir_mean,nir_mean,nir_mean,nir_mean,nir_mean,nir_mean,nir_mean
1,NIR_mean,NIR_mean,NIR_mean,NIR_mean,NIR_mean,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedi_2,_NIRmedi_1,_NIRmedi_1,_NIRmedian,_NIRmedian,NIR_June 6th,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedi_1,nir_median,nir_median,nir_median,nir_median,nir_median,nir_median,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedi_1,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedi_1,_NIRmedian,_NIRmedian,_NIRmedi_1,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedi_2,_NIRmedian,_NIRmedi_1,_NIRmedian,_NIRmedian,_NIRmedian,nir_median,nir_median,nir_median,nir_median,nir_median,nir_median,nir_median,nir_median
2,NIR_std,NIR_std,NIR_std,NIR_std,NIR_std,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstde_2,_NIRstde_1,_NIRstde_1,_NIRstdev,_NIRstdev,_NIRstdev_June 6th,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstde_1,nir_stdev,nir_stdev,nir_stdev,nir_stdev,nir_stdev,nir_stdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstde_1,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstde_1,_NIRstdev,_NIRstdev,_NIRstde_1,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstde_2,_NIRstdev,_NI

Missing values:  0
NIR => NIR column renamed in 76 of 76 found dataframes
col_df_non_std_cols_NIR


,Graminor_220719_corrected,Graminor_eastwest_250619_correct_duplicate_east,Masbasis_110619,Graminor_eastwest_090720_duplication_same_as_010720,Robot_Mica_070820,Robot_Mica_160720
0,NaN,_NIRmean,_NIRmean,_NIRmean,_NIRmean,NaN
1,NaN,_NIRmedian,_NIRmedian,_NIRmedian,_NIRmedian,NaN
2,NaN,_NIRstdev,_NIRstdev,_NIRstdev,_NIRstdev,NaN
3,NaN,_NIRmean.1,_NIRmean_1,_NIRmean.1,_NIRmean_1,NaN
4,NaN,_NIRmedian.1,_NIRmedi_1,_NIRmedian.1,_NIRmedi_1,NaN
5,NaN,_NIRstdev.1,_NIRstde_1,_NIRstdev.1,_NIRstde_1,NaN


Wall time: 1.43 s


### List of df with problems

In [18]:
%%time

list_problem_df_per_query
problem_df = []
for df in list_problem_df_per_query:
    problem_df.extend(locals()[df].columns)

print(len(set(problem_df)))
set(problem_df)

8
Wall time: 0 ns


{'Graminor_220719_corrected',
 'Graminor_eastwest_090720_duplication_same_as_010720',
 'Graminor_eastwest_250619_correct_duplicate_east',
 'Masbasis_110619',
 'Masbasis_mica_130721',
 'Robot_Mica_070820',
 'Robot_Mica_140820',
 'Robot_Mica_160720'}

Problems after including Masbasis 2021 and Staur 2019, 2020  
**Duplication of indices in data (no similarity in duplicate data)**  
 1. Graminor_eastwest_250619_correct_duplicate_east 
 2. Masbasis_110619  
 3. Robot_Mica_070820  
 4. Robot_Mica_140820  
   
**Duplication of indices in data (no similarity in duplicate data)**  
_One Half of the data is the same as in Graminor_east_010720_  
_Blue mean is in 3 columns where one is incomplete copy of the other while the third is different_  
 1. Graminor_eastwest_090720_duplication_same_as_010720  
   
**NIR Missing**  
 1. Graminor_220719_corrected  
 2. Robot_Mica_160720  
 3. Masbasis_mica_130721  

Probles after merging eastwest in Graminor  
 'Graminor_220719_corrected' NIR Missing  
 'Graminor_eastwest_090720_duplication-same_as_010720' duplication  
 'Graminor_eastwest_250619_correct_duplicate_east' duplication  
 'Masbasis_110619' duplication  
 'Robot_Mica_070820' duplication  
 'Robot_Mica_140820' duplication  
 'Robot_Mica_160720' NIR Missing  

Problem datasets before combining east west datasets in Graminor{'Graminor_220719_corrected',  
 'Graminor_east_090720'  
 'Graminor_east_250619_corrected' 
 'Graminor_west_020719'   
 'Masbasis_110619'   
 'Robot_Mica_070820'   
 'Robot_Mica_140820'   
 'Robot_Mica_160720'
 

Dataframes with Problems (begining)

 'Graminor_070819_2'  
 'Graminor_220719_corrected'  
 'Graminor_east_010720'  
 'Graminor_east_090720'  
 'Graminor_east_250619_corrected'  
 'Graminor_west_020719'  
 'Masbasis_Mica_130720'  
 'Robot_Mica_070820'  
 'Robot_Mica_140820'  
 'Robot_Mica_160720'  

## Standardise the column names
Get list of df with all required columns after implementing standard names 

### ToDo: Test: Check if there is duplication of column names

### List down standard column names

In [19]:
%%time

std_columns = []
std_columns_mean = []
std_columns_median = []
std_columns_stdev = []

for key, value in find_cols.items():
    if key in id_cols_var:
        std_columns.append(value)
        std_columns_mean.append(value)
        std_columns_median.append(value)
        std_columns_stdev.append(value)

    else:
        std_columns.extend([value+y for x, y in dtype_f.items()])
        std_columns_mean.append(value+dtype_f[0])
        std_columns_median.append(value+dtype_f[1])
        std_columns_stdev.append(value+dtype_f[2])
        
std_columns.remove('Plot_ID')
std_columns_mean.remove('Plot_ID')
std_columns_median.remove('Plot_ID')
std_columns_stdev.remove('Plot_ID')

std_columns
# std_columns_mean
# std_columns_median
# std_columns_stdev

Wall time: 0 ns


['Plot_ID',
 'Blue_Mean_Value',
 'Blue_Median_Value',
 'Blue_Std_Dev_Value',
 'Green_Mean_Value',
 'Green_Median_Value',
 'Green_Std_Dev_Value',
 'Red_Mean_Value',
 'Red_Median_Value',
 'Red_Std_Dev_Value',
 'RedEdge_Mean_Value',
 'RedEdge_Median_Value',
 'RedEdge_Std_Dev_Value',
 'NIR_Mean_Value',
 'NIR_Median_Value',
 'NIR_Std_Dev_Value']

In [20]:
find_cols

{'id': 'Plot_ID',
 'plot': 'Plot_ID',
 'Blue': 'Blue',
 'Gre': 'Green',
 'Red': 'Red',
 'RedE': 'RedEdge',
 'NIR': 'NIR'}

### Find df which have all the required std columns

In [21]:
%%time

complete_dataframes = []
incomplete_df = []

for df in all_df:
    df_columns = locals()[df].columns
    if (all(x in df_columns for x in std_columns)):
        complete_dataframes.append(df)
    else:
        incomplete_df.append(df)
print(f'Found {len(complete_dataframes)} complete dataframes. Can be accessed using \"complete_dataframes\" list')
print(f'Found {len(incomplete_df)} incomplete dataframes. Can be accessed using \"incomplete_df\" list')

Found 74 complete dataframes. Can be accessed using "complete_dataframes" list
Found 8 incomplete dataframes. Can be accessed using "incomplete_df" list
Wall time: 1.95 ms


## Exporting complete and incomplete datasets

In [22]:
# Exporting complete df
# Defining path
os.makedirs(complete_df_path, exist_ok=True)

# Test: Check if there are duplicate names in the complete_dataframes list
# As the last file with the same name will overwrite the previously exported file, resulting in data loss
if len(complete_dataframes) != len(set(complete_dataframes)):
    print('There are duplicate names in the complete_dataframes list.\nPlease check before exporting as it might result in data loss.')
        
    # Printing the names of the duplicate datasets, if any
    find_duplicates=[]
    for i in complete_dataframes:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\'')
    raise NameError
    
for df in complete_dataframes:
    locals()[df].to_csv(complete_df_path+df+'.csv', index=False)

In [23]:
# Defining path
os.makedirs(incomplete_df_path, exist_ok=True)

# Test: Check if there are duplicate names in the incomplete_df list
# As the last file with the same name will overwrite the previously exported file, resulting in data loss
if len(incomplete_df) != len(set(incomplete_df)):
    print('There are duplicate names in the incomplete_df list.\nPlease check before exporting as it might result in data loss.')
    
    # Printing the names of the duplicate datasets
    find_duplicates=[]
    for i in incomplete_df:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\'')
    raise NameError
    
for df in incomplete_df:
    locals()[df].to_csv(incomplete_df_path+df+'.csv', index=False)

In [24]:
import json
a_file = open("Data\std_columns.json", "w")
json.dump(std_columns, a_file)
a_file.close()

# a_file = open("Data\std_columns.json", "r")
# output = a_file.read()
# a_file.close()
# print(output)

# END OF SECTION